In [1]:
import cv2
import mediapipe as mp
import numpy as np
import random
import pandas as pd
import pickle
from utilities import *
from joint_angles import JointAngle, Angles
from body_parts import BodyPart

In [2]:
n = 8
angles = [0] * n
bp = BodyPart()

mpPose = mp.solutions.pose
video = '../assets/sample-video.mp4'
cap = cv2.VideoCapture(video)
filename = '../model/PhaseClassifier.pickle'
clf = pickle.load(open(filename, "rb"))
data = {
    'Angles': [],
    'Phase': []
}

In [3]:
with mpPose.Pose(min_detection_confidence=0.75, min_tracking_confidence=0.75) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image.flags.writeable = True
        
        try:
            lm = results.pose_landmarks.landmark
            jointAngle = JointAngle(lm)
            bp.cords = detect_joints(lm)
            angles = jointAngle.body_angles(bp)
            data['Angles'].append(angles)
            data['Phase'].append(None)
        except:
            pass

        # cv2.putText(
        #     image,
        #     'Down' if clf.predict([angles]) == 0 else 'Up',
        #     (0, 50),
        #     cv2.FONT_HERSHEY_SIMPLEX,
        #     1, (255,255,255), 2, cv2.LINE_AA
        # )

        # cv2.imshow('Video', image)
        # if cv2.waitKey(10) & 0xFF == ord('q'):
        #     break


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


error: OpenCV(4.8.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [4]:
df = pd.DataFrame(data)
features = df['Angles'].to_list()
features

[[1, 176, 177, 0, 180, 15, 177, 178],
 [2, 176, 177, 0, 180, 14, 178, 178],
 [2, 176, 177, 0, 180, 14, 179, 178],
 [2, 176, 177, 0, 179, 14, 179, 178],
 [2, 176, 177, 0, 179, 14, 179, 179],
 [2, 176, 177, 0, 179, 14, 179, 179],
 [2, 176, 177, 0, 179, 14, 179, 179],
 [2, 176, 177, 0, 179, 14, 179, 179],
 [2, 176, 177, 0, 179, 14, 179, 179],
 [2, 176, 177, 0, 179, 14, 179, 179],
 [2, 176, 177, 0, 179, 14, 179, 179],
 [2, 176, 177, 0, 179, 14, 179, 179],
 [2, 175, 177, 0, 179, 14, 179, 179],
 [2, 175, 177, 0, 179, 13, 180, 179],
 [2, 174, 177, 0, 179, 13, 180, 179],
 [2, 174, 176, 0, 179, 13, 180, 178],
 [2, 173, 176, 0, 179, 13, 180, 178],
 [2, 173, 176, 0, 179, 13, 180, 178],
 [2, 172, 175, 0, 179, 13, 180, 178],
 [2, 172, 175, 0, 179, 13, 180, 178],
 [2, 171, 174, 0, 179, 13, 180, 178],
 [2, 171, 173, 0, 179, 13, 180, 178],
 [2, 170, 172, 0, 179, 13, 180, 178],
 [2, 167, 170, 0, 179, 13, 180, 178],
 [2, 165, 169, 0, 179, 13, 179, 178],
 [2, 163, 167, 0, 179, 13, 179, 178],
 [2, 161, 16

In [5]:
labels = clf.predict(features)
df['Phase'] = labels
df.to_csv('../dataset/phases.csv')
df

,Angles,Phase
0,"[1, 176, 177, 0, 180, 15, 177, 178]",0
1,"[2, 176, 177, 0, 180, 14, 178, 178]",0
2,"[2, 176, 177, 0, 180, 14, 179, 178]",0
3,"[2, 176, 177, 0, 179, 14, 179, 178]",0
4,"[2, 176, 177, 0, 179, 14, 179, 179]",0
...,...,...
248,"[3, 175, 175, 0, 179, 14, 179, 179]",0
249,"[3, 175, 175, 0, 179, 13, 179, 179]",0
250,"[3, 175, 175, 0, 179, 14, 179, 179]",0
251,"[3, 175, 175, 0, 179, 14, 179, 179]",0


In [ ]:
clf.predict([features[0]])